In [1]:
import urllib.request
import datetime
import json

In [ ]:
client_id='아이디'
client_secret='시크릿'

In [3]:

def getRequestUrl(url):
    req= urllib.request.Request(url) # 전달 받은 url로 request 객체 생성

    # 네이버 인증용 id, secret 헤더에 추가
    req.add_header("X-Naver-Client-Id",client_id)
    req.add_header("X-Naver-Client-Secret",client_secret)

    try:
        response = urllib.request.urlopen(req) # api 요청 전송
        if response.getcode()==200: # 정상 응답(HTTP 200)
            return response.read().decode('utf-8')  # UTF-8로 디코딩된 JSON 문자열 반환
    except Exception as e:
        print(e)
        print('[%s] Error for URL : %s' %(datetime.datetime.now(),url))
        return None
    



In [4]:
def getNaverSearch(node, srcText, start, display):    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)
    
    url = base + node + parameters    
    responseDecode = getRequestUrl(url)   #[CODE 1]
    
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

In [5]:
def getPostData(post,jsonResult,cnt):
    title=post['title']
    description=post['originallink']
    org_link=post['link']
    
    pDate = datetime.datetime.strptime(post['pubDate'],  '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 
    'org_link':org_link,   'link': org_link,   'pDate':pDate})
    return    

In [6]:
def main():
    node = 'news'   # 크롤링 할 대상
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []

    jsonResponse = getNaverSearch(node, srcText, 1, 100)  #[CODE 2]
    total = jsonResponse['total']
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):         
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post, jsonResult, cnt)  #[CODE 3]       
        
        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100)  #[CODE 2]
       
    print('전체 검색 : %d 건' %total)
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult,  indent=4, sort_keys=True,  ensure_ascii=False)
                        
        outfile.write(jsonFile)
        
    print("가져온 데이터 : %d 건" %(cnt))
    print ('%s_naver_%s.json SAVED' % (srcText, node))


In [13]:

if __name__ == '__main__':  #임포트되지 않고 직접 실행하는 경우라면
    main()                  #main 함수 호출

HTTP Error 400: Bad Request
[2025-08-28 16:06:09.480118] Error for URL : https://openapi.naver.com/v1/search/news.json?query=naver&start=1001&display=100
전체 검색 : 1184070 건
가져온 데이터 : 1000 건
naver_naver_news.json SAVED


In [8]:
from urllib.parse import quote

srcText = "안녕하세요 Python!"
encoded = quote(srcText)
print(encoded)


%EC%95%88%EB%85%95%ED%95%98%EC%84%B8%EC%9A%94%20Python%21
